### Import libraries

In [1]:
import pandas as pd
import numpy as np
import holidays

### Read dataset

In [2]:
df = pd.read_csv('data/dataset_v2.csv')
df.head(5)

,offer_id,add_to_wishlist,add_to_cart,base_price_with_vat,ordered_quantity,orders,product_novelty,count_reviews,pageviews,rating,promo_price_with_vat,date,category_1,category_2,category_3
0,21,0,0,2009.99,0,0,2016-03-29,0,0,0.00,2009.99,2020-12-12,1,0,0
1,83,0,0,6699.99,0,0,2017-03-15,0,32,0.00,3999.99,2020-12-12,1,0,0
2,119,23,6,5799.99,3,3,2017-06-09,40,1218,4.35,4499.99,2020-12-12,1,0,0
3,120,13,1,5799.99,1,1,2017-06-09,21,350,4.52,4499.99,2020-12-12,1,0,0
4,121,1,0,7199.99,0,0,2017-06-12,0,26,0.00,5999.99,2020-12-12,1,0,0


### Feature Engineering

Let's do something about those datetime columns.

In [3]:
df['date'] = pd.to_datetime(df['date'])
df['product_novelty'] = pd.to_datetime(df['product_novelty'])
dates_diff = df.date - df.product_novelty

# Calculate differences
df['diff_days'] = dates_diff.dt.days
df['diff_years'] = df['date'].dt.year - df['product_novelty'].dt.year
df['diff_months'] = df['date'].dt.month - df['product_novelty'].dt.month + 12 * df['diff_years']
df.drop('product_novelty', axis=1, inplace=True)

Let's create some other temporal variables that will tell us if it's a weekend or a holiday.

In [4]:
# Define the Romanian holiday calendar for our dataframe
ro_holidays = holidays.Romania(years=[2020, 2021])

# Create a new column to indicate if it's a Romanian holiday
df['is_ro_holiday'] = df['date'].apply(lambda x: x in ro_holidays)
df['is_ro_holiday'] = np.where(df['is_ro_holiday'] == True, 1, 0)

# Flag weekends
df['is_weekend'] = df['date'].dt.weekday // 5

Calculating the conversion rate for each product.

In [5]:
df['conversion_rate'] = df['orders'] / df['pageviews']
df['conversion_rate'] = df['conversion_rate'].replace([np.inf, -np.inf], np.nan).fillna(0)

Compute the discount applied for each offer_id (absolute and percentage-based).

In [6]:
df['discount'] = df['base_price_with_vat'] - df['promo_price_with_vat']
df['discount_percentage'] = df['discount'] / df['base_price_with_vat']
df['discount_percentage'] = df['discount_percentage'].replace([np.inf, -np.inf], np.nan).fillna(0)

Proceed with rations to see how engaged the customers are.

In [7]:
# Creating the Wishlist to Pageviews Ratio
df['wishlist_to_pageviews_ratio'] = df['add_to_wishlist'] / df['pageviews'].replace(0, 1)
# Creating the Cart to Wishlist Ratio
df['cart_to_wishlist_ratio'] = df['add_to_cart'] / df['add_to_wishlist'].replace(0, 1)
# Creating the Orders to Cart Ratio
df['orders_to_cart_ratio'] = df['orders'] / df['add_to_cart'].replace(0, 1)
# Make sure we don't have any inf's
df[['wishlist_to_pageviews_ratio', 'cart_to_wishlist_ratio', 'orders_to_cart_ratio']] = df[['wishlist_to_pageviews_ratio', 'cart_to_wishlist_ratio', 'orders_to_cart_ratio']].replace([np.inf, -np.inf], np.nan).fillna(0)

Time to shift the target variable - pageviews.

In [8]:
df.sort_values(by=['offer_id', 'date'], inplace=True)
df['pageviews_-1d_lag'] = df.groupby('offer_id')['pageviews'].shift(-1)

Rolling statistics and lag variables.

In [9]:
# Columns for which to create rolling averages and lags
columns_to_process = ['add_to_wishlist', 'add_to_cart', 'orders']

# Creating 5-day rolling averages
for col in columns_to_process:
    df[f'{col}_5d_rolling_avg'] = df.groupby('offer_id')[col].transform(lambda x: x.rolling(window=5, min_periods=5).mean())

# Creating 1-day and 2-day lag variables
for col in columns_to_process:
    df[f'{col}_1d_lag'] = df.groupby('offer_id')[col].shift(1)
    df[f'{col}_2d_lag'] = df.groupby('offer_id')[col].shift(2)
    
df = df.dropna(axis=0).reset_index(drop=True)
df.drop(['pageviews'], axis=1, inplace=True)

For this problem, I have chosen not to create independent variables from the dependent variable for the following reasons:

- Model Versatility: This approach maintains flexibility in choosing various machine learning models, particularly those that might not handle time-dependent features effectively.
- Assumption of Independence: Many models assume feature independence. Using lagged versions of the target variable can violate this assumption, potentially impacting model performance.
- Avoiding Overfitting: Relying on past values of the dependent variable might lead to overfitting, especially in time series data, reducing the model's ability to generalize to new data.

### Export dataframe

In [10]:
df.to_csv('data/dataset_v3.csv', index=False)